In [ ]:
import pymc as pm
import arviz as az
import numpy as np


with pm.Model() as model:
    # Latent variable gamma
    gamma = pm.Bernoulli('gamma', p=0.5)
    
    # Input variables X and Y
    X = pm.Bernoulli('X', p=0.5)
    Y = pm.Bernoulli('Y', p=0.5)
    
    # Conditional probabilities for A and B given X, Y, gamma
    A = pm.Bernoulli('A', p=pm.math.switch(gamma, pm.math.switch(X, 0.9, 0.1), pm.math.switch(X, 0.2, 0.8)))
    B = pm.Bernoulli('B', p=pm.math.switch(gamma, pm.math.switch(Y, 0.9, 0.1), pm.math.switch(Y, 0.2, 0.8)))
    
    # Sample from the model
    trace = pm.sample(1000)

# Accessing the posterior samples
print(az.summary(trace, var_names=['A', 'B', 'X', 'Y']))


In [ ]:
# trace is the InferenceData object from sampling

# Extract the samples
samples = az.extract_dataset(trace, group='posterior', var_names=['A', 'B', 'X', 'Y'])
A_samples = samples.A.values.flatten()
B_samples = samples.B.values.flatten()
X_samples = samples.X.values.flatten()
Y_samples = samples.Y.values.flatten()

# Combine the samples into a single array of tuples
data = list(zip(A_samples, B_samples, X_samples, Y_samples))

# Create a dictionary to count occurrences of each (A, B, X, Y) combination
joint_distribution = {}

for entry in data:
    if entry in joint_distribution:
        joint_distribution[entry] += 1
    else:
        joint_distribution[entry] = 1

# Convert counts to probabilities by dividing by the total number of samples
total_samples = len(data)
for key in joint_distribution:
    joint_distribution[key] /= total_samples

# Print the joint distribution
print(joint_distribution)


In [ ]:
import numpy as np

# Extract samples from the trace
A_samples = trace.posterior['A'].values.flatten()
B_samples = trace.posterior['B'].values.flatten()
X_samples = trace.posterior['X'].values.flatten()
Y_samples = trace.posterior['Y'].values.flatten()

# Stack them into a single array for easier processing
samples = np.vstack((A_samples, B_samples, X_samples, Y_samples)).T

# Find unique rows and their counts
unique_rows, counts = np.unique(samples, axis=0, return_counts=True)

# Calculate probabilities by normalizing counts
probabilities = counts / counts.sum()

# Create a dictionary to store the joint distribution
joint_distribution = {}
for i, row in enumerate(unique_rows):
    key = tuple(row)  # Create a tuple key (A, B, X, Y)
    joint_distribution[key] = probabilities[i]

# Output the joint distribution
print(joint_distribution)


In [ ]:
import gurobipy as gp
import numpy as np

model = gp.Model("")

cardA, cardB, cardX, cardY, cardL = 2, 2, 2, 2, 8
P_ABXY = model.addMVar(shape=(cardA, cardB, cardX, cardY), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P_ABXY')

P_A = P_ABXY.sum(axis=(1, 2, 3))
P_B = P_ABXY.sum(axis=(0, 2, 3))
P_X = np.array([1/cardA for _ in range(cardA)]) # uniform distribution across setting options
P_Y = np.array([1/cardA for _ in range(cardA)]) # uniform distribution across setting options

##### creating a random distribution for P_l with given cardinality ######
rand_numbs = np.random.random(cardL)
normd_numbs = rand_numbs / np.sum(rand_numbs)
scaled_numbs = np.round(normd_numbs * 10**3) / 10**3
correction = 1 - np.sum(scaled_numbs)
largest_index = np.argmax(scaled_numbs)
scaled_numbs[largest_index] += correction

P_l = scaled_numbs
######


# NOT HOW THIS SHOULD BE DONE
P_A_giv_Xl = model.addMVar(shape=(cardA, cardX, cardL), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P(A|Xl)')
P_B_giv_Yl = model.addMVar(shape=(cardB, cardY, cardL), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P(B_Yl)')
model.update()




# # Define necessary conditional probabilities
# P_A_given_X = np.zeros((cardA, cardX))
# P_B_given_Y = np.zeros((cardB, cardY))
# P_Y_given_X = np.zeros((cardY, cardX))
# P_X_given_Y = np.zeros((cardX, cardY))
# P_X_given_A = np.zeros((cardA, cardX))
# P_Y_given_A = np.zeros((cardA, cardY))
# P_X_given_B = np.zeros((cardB, cardX))
# P_Y_given_B = np.zeros((cardB, cardY))
# for var1, var2 in np.ndindex(2,2):
#     P_A_given_X[var1, var2] = P_ABXY[var1, :, var2, :].sum() / P_X[var2]
#     P_B_given_Y[var1, var2] = P_ABXY[:, var1, :, var2].sum() / P_Y[var2]
#     P_Y_given_X[var1, var2] = P_ABXY[:, :, var2, var1].sum() / P_X[var2]
#     P_X_given_Y[var1, var2] = P_ABXY[:, :, var1, var2].sum() / P_Y[var2]
#     P_X_given_A[var1, var2] = P_ABXY[var2, :, var1, :].sum() / P_A[var2]
#     P_Y_given_A[var1, var2] = P_ABXY[var2, :, :, var1].sum() / P_A[var2]
#     P_X_given_B[var1, var2] = P_ABXY[:, var2, var1, :].sum() / P_B[var2]
#     P_Y_given_B[var1, var2] = P_ABXY[:, var2, :, var1].sum() / P_B[var2]
def P_A_given_X(a, x):
    return P_ABXY[a, :, x, :].sum() / P_X[x]
def P_B_given_Y(b, y):
    return P_ABXY[:, b, :, y].sum() / P_Y[y]
def P_Y_given_X(y, x):
    return P_ABXY[:, :, x, y].sum() / P_X[x]
def P_X_given_Y(x, y):
    return P_ABXY[:, :, x, y].sum() / P_Y[y]
def P_X_given_A(x, a):
    return P_ABXY[a, :, x, :].sum() / P_A[a]
def P_Y_given_A(y, a):
    return P_ABXY[a, :, :, y].sum() / P_A[a]
def P_X_given_B(x, b):
    return P_ABXY[:, b, x, :].sum() / P_B[b]
def P_Y_given_B(y, b):
    return P_ABXY[:, b, :, y].sum() / P_B[b]
"""
for x,a in np.ndindex(cardX, cardA):
    P_A_given_X[a, x] = P_ABXY[a, :, x, :].sum() / P_X[x]
for y,b in np.ndindex(cardY, cardB):
    P_B_given_Y[b, y] = P_ABXY[:, b, :, y].sum() / P_Y[y]
for x,y in np.ndindex(cardX, cardY):
    P_Y_given_X[y, x] = P_ABXY[:, :, x, y].sum() / P_X[x]
for y,x in np.ndindex(cardY, cardX):
    P_X_given_Y[x, y] = P_ABXY[:, :, x, y].sum() / P_Y[y]
for a,x in np.ndindex(cardA, cardX):
    P_X_given_A[a, x] = P_ABXY[a, :, x, :].sum() / P_A[a]
for a,y in np.ndindex(cardA, cardY):
    P_Y_given_A[a, y] = P_ABXY[a, :, :, y].sum() / P_A[a]
for b,x in np.ndindex(cardB, cardX):
    P_X_given_B[b, x] = P_ABXY[:, b, x, :].sum() / P_B[b]
for b,y in np.ndindex(cardB, cardY):
    P_Y_given_B[b, y] = P_ABXY[:, b, :, y].sum() / P_B[b]
"""

# SEM
for a,b,x,y in np.ndindex(cardA, cardB, cardX, cardY):
    # model.addConstr(P_AB_giv_XY[a,b,x,y] == [P_A_giv_Xl[a,x,l] * P_B_giv_Yl(b,y,l)*P_l[l] for l in range(cardL)].sum())
    model.addConstr(P_ABXY[a,b,x,y] == sum([P_A_giv_Xl[a,x,l] * P_B_giv_Yl[b,y,l]*P_X[x]*P_Y[y]*P_l[l] for l in range(cardL)]))


# conditional independence constraints for bell DAG

# X ⫫ Y 
# -> P(X, Y) = P(X)P(Y)
for x,y in np.ndindex(cardX, cardY):
    model.addConstr(P_ABXY[:, :, x, y].sum() == P_X[x] * P_Y[y], name='X⫫Y')
    
# A ⫫ Y | X 
# -> P(A, Y | X) = P(A | X)P(Y | X)
for a,x,y in np.ndindex(cardA, cardX, cardY):
    model.addConstr(
        # P(A,X,Y)/P(X) = P(A,Y|X) = P(A|X)P(Y|X)
        P_ABXY[a, :, x, y].sum() / P_X[x] == P_A_given_X[a, x] * P_Y_given_X[y, x], name=f'P(A,Y|X)=P(A|X)P(Y|X)')

# B ⫫ X | Y 
# -> P(B, X | Y) = P(B | Y)P(X | Y)
for b,x,y in np.ndindex(cardB,cardX,cardY):
    model.addConstr(
        # P(B,X,Y)/P(Y) = P(B,X|Y) = P(B|Y)P(X|Y)
        P_ABXY[:, b, x, y].sum() / P_Y[y] == P_B_given_Y[b, y] * P_X_given_Y[x, y], name=f'P(B,X|Y)=P(B|Y)P(X|Y)')

# A ⫫ Y 
# -> P(A, Y) = P(A)P(Y)
for a,y in np.ndindex(cardA, cardY):
    model.addConstr(P_ABXY[a, :, :, y].sum() == P_A[a] * P_Y[y], name=f'A⫫Y_{a}_{y}')

# B ⫫ X 
# -> P(B,X) = P(B)P(X)
for b,x in np.ndindex(cardB, cardX):
    model.addConstr(P_ABXY[:, b, x, :].sum() == P_B[b] * P_X[x], name=f'B⫫X_{b}_{x}')

for a,x,y in np.ndindex(cardA, cardX, cardY):
    joint_prob = P_ABXY[a, :, x, y].sum() / P_A[a] # P(X,Y|A) = P(X|A)P(Y|A)
    model.addConstr(joint_prob == P_X_given_A[a, x] * P_Y_given_A[a, y], name=f'P(X,Y|A) = P(X|A)P(Y|A)')

# X ⫫ Y | B -> P(X, Y | B) = P(X | B)P(Y | B) and = P(X)P(Y | B)
for b,x,y in np.ndindex(cardB, cardX, cardY):
    joint_prob = P_ABXY[:, b, x, y].sum() / P_B[b] # P(X,Y|B) = P(X|B)P(Y|B)
    model.addConstr(joint_prob == P_X_given_B[b, x] * P_Y_given_B[b, y], name=f'P(X,Y|B) = P(X|B)P(Y|B)')



# normalization constraints:
model.addConstr(P_A.sum() == 1, name='P_A_sum')
model.addConstr(P_B.sum() == 1, name='P_B_sum')
model.addConstr(P_X.sum() == 1, name='P_X_sum')
model.addConstr(P_Y.sum() == 1, name='P_Y_sum')
model.addConstr(P_l.sum() == 1, name='P_l_sum')
model.addConstr(P_ABXY.sum() == 1, name='P_ABXY_sum')


model.optimize()

In [ ]:
# import pandas as pd
# import numpy as np
# from scipy.stats import distributions
# import networkx as nx

# def sim_from_dag(dag, n_sim, sort_dag=True, check_inputs=True):
#     if check_inputs:
#         check_inputs_sim_from_dag(dag, n_sim, sort_dag)

#     # Sample from root nodes
#     data = {}
#     for node in dag['root_nodes']:
#         # Add n_sim to existing arguments
#         args = node['params']
#         args['size'] = n_sim

#         # Call data generation function
#         try:
#             dist_function = getattr(distributions, node['type'])
#             out = pd.DataFrame(dist_function(**args), columns=[node['name']])
#         except Exception as e:
#             raise Exception(f"An error occurred when processing root node '{node['name']}'. The message was: {e}")
        
#         data[node['name']] = out[node['name']]

#     data = pd.DataFrame(data)

#     if not dag['child_nodes']:
#         return data

#     # If not already ordered properly, use topological sorting to get the right data generation sequence
#     if sort_dag:
#         adjacency_mat = dag2matrix(dag, include_root_nodes=False)
#         G = nx.DiGraph(adjacency_mat)
#         index_children = list(nx.topological_sort(G))
#     else:
#         index_children = list(range(len(dag['child_nodes'])))

#     # Go through DAG step by step
#     for i in index_children:
#         child = dag['child_nodes'][i]
#         args = child.copy()
#         args['data'] = data

#         if child['type'] != "cox":
#             args.pop('name', None)

#         try:
#             node_function = globals()[f"node_{child['type']}"]
#             node_out = node_function(**args)
#             data = add_node_to_data(data, node_out, child['name'])
#         except Exception as e:
#             raise Exception(f"An error occurred when processing node '{child['name']}'. The message was: {e}")

#     return data

# # Auxiliary function examples
# def node_example(data, **kwargs):
#     # Example of a function for a node processing
#     pass

# def add_node_to_data(data, new, name):
#     # Add new node results to data
#     data[name] = new
#     return data

# # Function to convert DAG to adjacency matrix (assuming some structure of 'dag')
# def dag2matrix(dag, include_root_nodes):
#     # Convert a dag to an adjacency matrix
#     pass


In [ ]:
import gurobipy as gp
import numpy as np

# distribution over a dag with all observable variables, then marginalize over the latent variables to get the joint distribution as if those are latent.
model = gp.Model("")

cardA, cardB, cardX, cardY, cardL = 2, 2, 2, 2, 8
P_ABXYL = model.addMVar(shape=(cardA, cardB, cardX, cardY, cardL), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P_ABXY')

P_A = P_ABXY.sum(axis=(1, 2, 3))
P_B = P_ABXY.sum(axis=(0, 2, 3))
P_X = np.array([1/cardA for _ in range(cardA)]) # uniform distribution across setting options
P_Y = np.array([1/cardA for _ in range(cardA)])
# P_l is an array of cardL random values that sum to 1
P_L = np.array([1/cardL for _ in range(cardL)])

# NOT HOW THIS SHOULD BE DONE
P_A_giv_Xl = model.addMVar(shape=(cardA, cardX, cardL), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P(A|Xl)')
P_B_giv_Yl = model.addMVar(shape=(cardB, cardY, cardL), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='P(B_Yl)')
model.update()

# Define necessary conditional probabilities
P_A_given_X = np.zeros((cardA, cardX))
P_B_given_Y = np.zeros((cardB, cardY))
P_Y_given_X = np.zeros((cardY, cardX))
P_X_given_Y = np.zeros((cardX, cardY))
P_X_given_A = np.zeros((cardA, cardX))
P_Y_given_A = np.zeros((cardA, cardY))
P_X_given_B = np.zeros((cardB, cardX))
P_Y_given_B = np.zeros((cardB, cardY))



# for var1, var2 in np.ndindex(2,2):
#     P_A_given_X[var1, var2] = P_ABXY[var1, :, var2, :].sum() / P_X[var2]
#     P_B_given_Y[var1, var2] = P_ABXY[:, var1, :, var2].sum() / P_Y[var2]
#     P_Y_given_X[var1, var2] = P_ABXY[:, :, var2, var1].sum() / P_X[var2]
#     P_X_given_Y[var1, var2] = P_ABXY[:, :, var1, var2].sum() / P_Y[var2]
#     P_X_given_A[var1, var2] = P_ABXY[var2, :, var1, :].sum() / P_A[var2]
#     P_Y_given_A[var1, var2] = P_ABXY[var2, :, :, var1].sum() / P_A[var2]
#     P_X_given_B[var1, var2] = P_ABXY[:, var2, var1, :].sum() / P_B[var2]
#     P_Y_given_B[var1, var2] = P_ABXY[:, var2, :, var1].sum() / P_B[var2]
def P_A_given_X(a, x):
    return P_ABXY[a, :, x, :].sum() / P_X[x]
def P_B_given_Y(b, y):
    return P_ABXY[:, b, :, y].sum() / P_Y[y]
def P_Y_given_X(y, x):
    return P_ABXY[:, :, x, y].sum() / P_X[x]
def P_X_given_Y(x, y):
    return P_ABXY[:, :, x, y].sum() / P_Y[y]
def P_X_given_A(x, a):
    return P_ABXY[a, :, x, :].sum() / P_A[a]
def P_Y_given_A(y, a):
    return P_ABXY[a, :, :, y].sum() / P_A[a]
def P_X_given_B(x, b):
    return P_ABXY[:, b, x, :].sum() / P_B[b]
def P_Y_given_B(y, b):
    return P_ABXY[:, b, :, y].sum() / P_B[b]
"""
for x,a in np.ndindex(cardX, cardA):
    P_A_given_X[a, x] = P_ABXY[a, :, x, :].sum() / P_X[x]
for y,b in np.ndindex(cardY, cardB):
    P_B_given_Y[b, y] = P_ABXY[:, b, :, y].sum() / P_Y[y]
for x,y in np.ndindex(cardX, cardY):
    P_Y_given_X[y, x] = P_ABXY[:, :, x, y].sum() / P_X[x]
for y,x in np.ndindex(cardY, cardX):
    P_X_given_Y[x, y] = P_ABXY[:, :, x, y].sum() / P_Y[y]
for a,x in np.ndindex(cardA, cardX):
    P_X_given_A[a, x] = P_ABXY[a, :, x, :].sum() / P_A[a]
for a,y in np.ndindex(cardA, cardY):
    P_Y_given_A[a, y] = P_ABXY[a, :, :, y].sum() / P_A[a]
for b,x in np.ndindex(cardB, cardX):
    P_X_given_B[b, x] = P_ABXY[:, b, x, :].sum() / P_B[b]
for b,y in np.ndindex(cardB, cardY):
    P_Y_given_B[b, y] = P_ABXY[:, b, :, y].sum() / P_B[b]
"""

# SEM
for a,b,x,y in np.ndindex(cardA, cardB, cardX, cardY):
    # model.addConstr(P_AB_giv_XY[a,b,x,y] == [P_A_giv_Xl[a,x,l] * P_B_giv_Yl(b,y,l)*P_l[l] for l in range(cardL)].sum())
    model.addConstr(P_ABXY[a,b,x,y] == sum([P_A_giv_Xl[a,x,l] * P_B_giv_Yl[b,y,l]*P_X[x]*P_Y[y]*P_l[l] for l in range(cardL)]))


# conditional independence constraints for bell DAG

# X ⫫ Y 
# -> P(X, Y) = P(X)P(Y)
for x,y in np.ndindex(cardX, cardY):
    model.addConstr(P_ABXY[:, :, x, y].sum() == P_X[x] * P_Y[y], name='X⫫Y')
    
# A ⫫ Y | X 
# -> P(A, Y | X) = P(A | X)P(Y | X)
for a,x,y in np.ndindex(cardA, cardX, cardY):
    model.addConstr(
        # P(A,X,Y)/P(X) = P(A,Y|X) = P(A|X)P(Y|X)
        P_ABXY[a, :, x, y].sum() / P_X[x] == P_A_given_X[a, x] * P_Y_given_X[y, x], name=f'P(A,Y|X)=P(A|X)P(Y|X)')

# B ⫫ X | Y 
# -> P(B, X | Y) = P(B | Y)P(X | Y)
for b,x,y in np.ndindex(cardB,cardX,cardY):
    model.addConstr(
        # P(B,X,Y)/P(Y) = P(B,X|Y) = P(B|Y)P(X|Y)
        P_ABXY[:, b, x, y].sum() / P_Y[y] == P_B_given_Y[b, y] * P_X_given_Y[x, y], name=f'P(B,X|Y)=P(B|Y)P(X|Y)')

# A ⫫ Y 
# -> P(A, Y) = P(A)P(Y)
for a,y in np.ndindex(cardA, cardY):
    model.addConstr(P_ABXY[a, :, :, y].sum() == P_A[a] * P_Y[y], name=f'A⫫Y_{a}_{y}')

# B ⫫ X 
# -> P(B,X) = P(B)P(X)
for b,x in np.ndindex(cardB, cardX):
    model.addConstr(P_ABXY[:, b, x, :].sum() == P_B[b] * P_X[x], name=f'B⫫X_{b}_{x}')

for a,x,y in np.ndindex(cardA, cardX, cardY):
    joint_prob = P_ABXY[a, :, x, y].sum() / P_A[a] # P(X,Y|A) = P(X|A)P(Y|A)
    model.addConstr(joint_prob == P_X_given_A[a, x] * P_Y_given_A[a, y], name=f'P(X,Y|A) = P(X|A)P(Y|A)')

# X ⫫ Y | B -> P(X, Y | B) = P(X | B)P(Y | B) and = P(X)P(Y | B)
for b,x,y in np.ndindex(cardB, cardX, cardY):
    joint_prob = P_ABXY[:, b, x, y].sum() / P_B[b] # P(X,Y|B) = P(X|B)P(Y|B)
    model.addConstr(joint_prob == P_X_given_B[b, x] * P_Y_given_B[b, y], name=f'P(X,Y|B) = P(X|B)P(Y|B)')



# normalization constraints:
model.addConstr(P_A.sum() == 1, name='P_A_sum')
model.addConstr(P_B.sum() == 1, name='P_B_sum')
model.addConstr(P_X.sum() == 1, name='P_X_sum')
model.addConstr(P_Y.sum() == 1, name='P_Y_sum')
model.addConstr(P_l.sum() == 1, name='P_l_sum')
model.addConstr(P_ABXY.sum() == 1, name='P_ABXY_sum')


model.optimize()

In [ ]:
print(P_l)
P_A = P_ABXYl.sum(axis=(1, 2, 3, 4))
P_B = P_ABXYl.sum(axis=(0, 2, 3, 4))


P_A_giv_X = gp.tupledict()
P_B_giv_Y = gp.tupledict()
P_Y_giv_X = gp.tupledict()
P_X_giv_Y = gp.tupledict()
P_A_giv_Xl = gp.tupledict()
P_B_giv_Yl = gp.tupledict()
for var1, var2 in np.ndindex(2,2):
    P_A_giv_X[var1, var2] = P_ABXYl[var1,:,var2,:,:].sum() / P_X[var2]
    P_B_giv_Y[var1, var2] = P_ABXYl[:,var1,:,var2,:].sum() / P_Y[var2]
    P_Y_giv_X[var1, var2] = P_ABXYl[:,:,var2,var1,:].sum() / P_X[var2]
    P_X_giv_Y[var1, var2] = P_ABXYl[:,:,var1,var2,:].sum() / P_Y[var2]

for a,x,l in np.ndindex(cardA, cardX, cardL): # P(A|X,l) from P(A,B,X,Y,l)
    P_A_giv_Xl[a,x,l] = P_ABXYl[a,:,x,:,l].sum() / (P_X[x] * P_l[l]) # P(A|X)/ (P(x)*P(l))
for b,y,l in np.ndindex(cardB, cardY, cardL): # P(B|Y,l) from P(A,B,X,Y,l)
    P_B_giv_Yl[b,y,l] = P_ABXYl[:,b,:,y,l].sum() / (P_Y[y] * P_l[l]) # P(B|Y)/ (P(B)*P(l))

# P(X)*P(L) = P(X,L)
for x,l in np.ndindex(cardX, cardL):
    model.addConstr(P_ABXYl[:,:,x,:,l].sum() == P_ABXYl[:,:,x,:,:].sum() * P_ABXYl[:,:,:,:,l].sum(), name='P(X)*P(L) = P(X,L)')
# P(Y)*P(L) = P(Y,L)
for y,l in np.ndindex(cardY, cardL):
    model.addConstr(P_ABXYl[:,:,:,y,l].sum() == P_ABXYl[:,:,:,y,:].sum() * P_ABXYl[:,:,:,:,l].sum() , name='P(Y)*P(L) = P(Y,L)')

## SEM
# for a,b,x,y in np.ndindex(cardA, cardB, cardX, cardY):
#     model.addConstr(P_ABXYl[a,b,x,y] == sum([P_A_giv_Xl[a,x,l] * P_B_giv_Yl[b,y,l]*P_X[x]*P_Y[y]*P_l[l] for l in range(cardL)]))
for a,b,x,y,l in np.ndindex(*P_ABXYl.shape):
    P_ABXYl[a,b,x,y,l] == P_A_giv_Xl[a,x,l] * P_B_giv_Yl[b,y,l]*P_X[x]*P_Y[y]*P_l[l]


##### conditional independence constraints for bell DAG
# X ⫫ Y 
# -> P(X, Y) = P(X)P(Y)
for x,y in np.ndindex(cardX, cardY):
    model.addConstr(P_ABXYl[:, :, x, y,:].sum() == P_X[x] * P_Y[y], name='X⫫Y')
    
# A ⫫ Y | X 
# -> P(A, Y | X) = P(A | X)P(Y | X)
for a,x,y in np.ndindex(cardA, cardX, cardY):
    model.addConstr(
        # P(A,X,Y)/P(X) = P(A,Y|X) = P(A|X)P(Y|X)
        P_ABXYl[a, :, x, y,:].sum() / P_X[x] == P_A_giv_X[a, x] * P_Y_giv_X[y, x], name=f'P(A,Y|X)=P(A|X)P(Y|X)')

# B ⫫ X | Y 
# -> P(B, X | Y) = P(B | Y)P(X | Y)
for b,x,y in np.ndindex(cardB,cardX,cardY):
    model.addConstr(
        # P(B,X,Y)/P(Y) = P(B,X|Y) = P(B|Y)P(X|Y)
        P_ABXYl[:, b, x, y,:].sum() / P_Y[y] == P_B_giv_Y[b, y] * P_X_giv_Y[x, y], name=f'P(B,X|Y)=P(B|Y)P(X|Y)')

# A ⫫ Y 
# -> P(A, Y) = P(A)P(Y)
for a,y in np.ndindex(cardA, cardY):
    model.addConstr(P_ABXYl[a, :, :, y,:].sum() == P_A[a] * P_Y[y], name=f'A⫫Y_{a}_{y}')

# B ⫫ X 
# -> P(B,X) = P(B)P(X)
for b,x in np.ndindex(cardB, cardX):
    model.addConstr(P_ABXYl[:, b, x, :,:].sum() == P_B[b] * P_X[x], name=f'B⫫X_{b}_{x}')

# for a,x,y in np.ndindex(cardA, cardX, cardY):
#     joint_prob = P_ABXYl[a, :, x, y,:].sum() / P_A[a] # P(X,Y|A) = P(X|A)P(Y|A)
#     model.addConstr(joint_prob == P_X_giv_A[a, x] * P_Y_giv_A[a, y], name=f'P(X,Y|A) = P(X|A)P(Y|A)')

# # X ⫫ Y | B -> P(X, Y | B) = P(X | B)P(Y | B) and = P(X)P(Y | B)
# for b,x,y in np.ndindex(cardB, cardX, cardY):
#     joint_prob = P_ABXYl[:, b, x, y,:].sum() / P_B[b] # P(X,Y|B) = P(X|B)P(Y|B)
#     model.addConstr(joint_prob == P_X_giv_B[b, x] * P_Y_giv_B[b, y], name=f'P(X,Y|B) = P(X|B)P(Y|B)')


# normalization constraints:
model.addConstr(P_A.sum() == 1, name='P_A_sum')
model.addConstr(P_B.sum() == 1, name='P_B_sum')
model.addConstr(P_ABXYl.sum() == 1, name='P_ABXYl_sum')
## need to write more generally:
for j in range(2):
    model.addConstr(sum([P_A_giv_X[i,j] for i in range(2)])==1)
    model.addConstr(sum([P_B_giv_Y[i,j] for i in range(2)])==1)
    model.addConstr(sum([P_Y_giv_X[i,j] for i in range(2)])==1)
    model.addConstr(sum([P_X_giv_Y[i,j] for i in range(2)])==1)
for j,k in np.ndindex(2,2):
    model.addConstr(sum([P_A_giv_Xl[i,j,k] for i in range(2)])==1)
    model.addConstr(sum([P_B_giv_Yl[i,j,k] for i in range(2)])==1)  
##


model.optimize()